# Сравнение обученных моделей и выбор основы чат-бота

На предыдущем этапе мною была обучена модель flan-t5-base


На этапе обучения на валидационной выборке рассчитывалась метрика rouge и bertscore для оценки похожести сгенерированного текста с ответом из сценария

Хотя данная метрика используется для оценки качества суммаризации, было интересно посмотреть как она сработает в случае диалоговой системы. Также она позволяет сравнивать модели между собой вовремя обучения. Я решила дополнить эту оценку косинусоной близостью с эталонным ответом (воспользуюсь моделью би-энкодера, обученного на предыдущей задаче) и насколько сгенерированный ответ подходит для вопроса и котекста (воспользуюсь моделью reranker, обученного на предыдущей задаче)

In [2]:
import pandas as pd
from utils import encode, cosine_sim
import warnings
import wandb
import json
import json
import time
import torch
import logging

warnings.filterwarnings("ignore")

import os
import json
import time
import torch
import logging

# Подготовим тестовую выборку

In [3]:
df = pd.read_pickle("data/scripts_reworked.pkl")
df.head(5)

,answer,question,context
0,"There’s no point, I just think it’s a good id...","Agreed, what’s your point?",
1,I think this is the place.,"If you have to ask, maybe you shouldn’t be here.","Hang on. One across is Aegean, eight down is..."
2,I think this is the place.,"If you have to ask, maybe you shouldn’t be here.","One across is Aegean, eight down is Nabakov, ..."
3,I think this is the place.,"If you have to ask, maybe you shouldn’t be here.","Can I help you? Yes. Um, is this the High IQ ..."
4,I think this is the place.,"If you have to ask, maybe you shouldn’t be here.","Yes. Um, is this the High IQ sperm bank?"


In [4]:
test = df.sample(n=30)

# Подходы к генерации текста

Первым шагом необходимо сгенерировать ответы 
Для генерации воспользуемся стратегией семплирования и выберем в качестве сравниваемых параметров top_p + temperature. 

Создадим функцию для генерации, которая будет принимать модель, ее токенайзер, знак зодиака для генерация и оба параметра. Эту функцию будем использовать в другой функции при оценке генерируемых текстов.

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
def generate_response(model,model_name, tokenizer, question, context, top_p, temperature):
    combined = "context: " + context + "</s>" + "question: " + question
    input_ids = tokenizer.encode(combined, return_tensors="pt")
    sample_output = model.generate(
        input_ids.to(device),
        do_sample=True,
        max_length=1000,
        top_p=top_p,
        temperature=temperature,
        repetition_penalty=2.0,
        top_k=50,
        no_repeat_ngram_size=2,
        early_stopping=True,
    )
    if model_name == "T5":
        return tokenizer.decode(sample_output[0], skip_special_tokens=True)
    else:
        out = tokenizer.decode(sample_output[0][1:], skip_special_tokens=True)
        if "</s>" in out:
            out = out[: out.find("</s>")].strip()
        return out

# Подходы к оценке текста
Разные модели и стратегии генерации будем сравнивать и оценивать на сходство с эталонным ответом, который есть в сценарии на основе небольшой случайной выборки из данных:

1. Сходство с оригиналом по косинусной близости. 
2. Время генерации ответа - бот будет работать на cpu, поэтому сравнительное время интересно посомотреть (относительные параметры не показательны, так как бот будет разворачиваться на разных машинах)

In [8]:
from sentence_transformers import SentenceTransformer

ranking_model = SentenceTransformer("Shakhovak/chatbot_sentence-transformer")

In [10]:
with open("config.json", "r") as f:
    json_config = json.load(f)
TOKEN = json_config["token"]
wandb.login(key=TOKEN)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: katya_shakhova (shakhova). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Kate\.netrc


True

In [11]:
def review_gen(test_df, model, model_name, tokenizer, top_p, temperature):
    print_top_p = str(int(top_p*10))
    print_temperature = str(int(temperature * 10))

    generated_results = pd.DataFrame()

    run = wandb.init(
        project="generative_models_chat",
        name=f"{model_name}_top_p={print_top_p}_temp={print_temperature}",
    )
    wandb.define_metric("gen_time", summary="mean")
    wandb.define_metric("cosine_scores", summary="mean")

    for index, row in test_df.iterrows():

        start = time.time()
        gen_text = generate_response(
                model=model,
                model_name=model_name,
                question=row['question'],
                context=row['context'], 
                tokenizer=tokenizer,           
                top_p=top_p,
                temperature=temperature,
            )

        logging.info(f"Generation for model {model_name} finished!")

        end = time.time()
        gen_time = round(end - start, 4)

        # check context format!!!
        answer_true_encoding = encode(
            texts=row['answer'],
            model=ranking_model,
            contexts=row['context'],
        )

        answer_generated_encoding = encode(
            texts=gen_text,
            model=ranking_model,
            contexts=row["context"],
        )

        cosine_scores = cosine_sim(
            answer_true_encoding,
            answer_generated_encoding,
        )
        wandb.log(
            {
                "gen_time": gen_time,
                "cosine_scores": cosine_scores,
            }
            )

        new_row = {
            f"{model_name}_gen_answer": gen_text,
            "answer_true": row['answer'],
            "question":row['question'],
            "context": row['context'],
            f"{model_name}_gen_time": gen_time,
            f"{model_name}_cos_sim_biencoder": cosine_scores,
        }

        generated_results = pd.concat([generated_results, pd.DataFrame([new_row])])

    data_to_log = wandb.Table(dataframe=generated_results)
    run.log({f"{model_name}_top_p={print_top_p}_temp={print_temperature}": data_to_log})

    logging.info(f"Artifacts for model {model_name} loaded to wandb!")

    wandb.finish()

# Flan-t5-base v2

In [12]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_flan = AutoTokenizer.from_pretrained("Shakhovak/flan-t5-base-sheldon-chat-v2")
model_flan = AutoModelForSeq2SeqLM.from_pretrained(
    "Shakhovak/flan-t5-base-sheldon-chat-v2"
)
model_flan.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [13]:
parameters_combinations = [(0.2,0.1),(0.7, 0.8),(0.5,0.5), (1, 0.95)]

In [14]:
for i in parameters_combinations:
    review_gen(test_df=test, 
                model=model_flan,
                model_name='flan',
                tokenizer=tokenizer_flan,
                top_p=i[1], temperature=i[0])

cosine_scores,▅▅███▇▆▆▆▄▂▅▄▅▆▇▆▇▂▃█▇█▁█▆██▄▄
gen_time,█▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


cosine_scores,▅▅███▇▅▆▇▅▂▆▄▄▆▇▂▇▂▃▆▆█▁█▁▇█▄▆
gen_time,▇▁▃▃▂▅▅▁▁█▂▃▃▄▂▂▄▃▃▅▆▁▆▃▄▂▄▅▃▁


cosine_scores,▅▄███▁▅▆▆▃▁▅▃▄▁▇▁▇▁▂█▇███▆█▇▃▃
gen_time,▂▁▅▅▃▂▅▂▃▅▅▃▅▅▅▃▆▅▂▅█▁▄▂█▆▆▅▆▅


cosine_scores,▆▅▄▅▇▁▆▆▆▄▁▄▆▆▆▄▆▇▁▁█▇▆▇█▄█▂▆▃
gen_time,▂▁▃▃▅▄▂▁▄▆▅▁▃▆▁▂▃▄▃█▂▁▂▄▃▃▂▃▂▁
